In [1]:
protein_groups_file = "../../test_data/system_tests/quicktests/mq_evidence/proteinGroups.txt.shortened.txt"
protein_groups_file_double = f"{protein_groups_file}.double.txt"
evidence_file = "../../test_data/system_tests/quicktests/mq_evidence/shortened_input.tsv"
annotated_file = f"{evidence_file}.protgroup_annotated.tsv"
aq_reformatted_file = "../../test_data/system_tests/quicktests/mq_evidence/shortened_input.tsv.protgroup_annotated.tsv.maxquant_evidence.aq_reformat.tsv"

In [10]:
import directlfq.utils


directlfq.utils.add_mq_protein_group_ids_if_applicable_and_obtain_annotated_file(mq_file=evidence_file, mq_protein_group_file=protein_groups_file, input_type_to_use="maxquant_evidence", columns_to_add=[])

'../../test_data/system_tests/quicktests/mq_evidence/shortened_input.tsv.protgroup_annotated.tsv'

In [11]:
import pandas as pd
protein_groups_df = pd.read_csv(protein_groups_file, sep="\t")
evidence_df = pd.read_csv(evidence_file, sep="\t")
annotated_df = pd.read_csv(annotated_file, sep="\t")


In [13]:
import directlfq.utils

def assert_that_evidence_df_scales_w_number_of_proteins(evidence_file, protein_groups_file, protein_groups_file_double):
    evidence_file_annotated = f"{evidence_file}.protgroup_annotated.tsv"
    directlfq.utils.add_mq_protein_group_ids_if_applicable_and_obtain_annotated_file(mq_file=evidence_file, mq_protein_group_file=protein_groups_file,  input_type_to_use="maxquant_evidence", columns_to_add=[])
    evidence_df_annotated = pd.read_csv(evidence_file_annotated, sep="\t")
    #display(evidence_df_annotated)
    length_annotated_single = len(evidence_df_annotated.index)
    directlfq.utils.add_mq_protein_group_ids_if_applicable_and_obtain_annotated_file(mq_file=evidence_file, mq_protein_group_file=protein_groups_file_double,  input_type_to_use="maxquant_evidence", columns_to_add=[])
    evidence_df_annotated = pd.read_csv(evidence_file_annotated, sep="\t")
    #display(evidence_df_annotated)
    length_annotated_double = len(evidence_df_annotated.index)
    assert length_annotated_double == length_annotated_single * 2

assert_that_evidence_df_scales_w_number_of_proteins(evidence_file, protein_groups_file, protein_groups_file_double)



In [14]:
import directlfq.utils
import pandas as pd
import numpy as np
def compare_number_of_datapoints_in_traces_w_protgroup_id_mapping(ion_file, protein_groups_file, evidence_file):
    ion_df = load_mq_dataframe(ion_file)
    protein2num_peptides_proteingroups_df = get_protein2num_peptides_proteingroups_df(protein_groups_file, evidence_file)
    protein2num_peptides_ion_df = get_protein2numpeptides_ion_df(ion_df)
    for protid in protein2num_peptides_ion_df.keys():
        num_ions_iondf = protein2num_peptides_ion_df[protid]
        num_ions_proteingroupsdf = protein2num_peptides_proteingroups_df[protid]
        print(f"protid: {protid}, num_ions_iondf: {num_ions_iondf}, num_ions_proteingroupsdf: {num_ions_proteingroupsdf}")
        assert num_ions_iondf == num_ions_proteingroupsdf

def load_mq_dataframe(example_mq_file):
    mq_df = pd.read_csv(example_mq_file, sep = "\t")
    mq_df = directlfq.utils.index_and_log_transform_input_df(mq_df)
    mq_df = mq_df.reindex(sorted(mq_df.columns, key= lambda x : get_fraction_number(x)), axis=1)
    return mq_df

def get_fraction_number(fraction_name):
    shortened_fraction = fraction_name.split("_")[1].replace("K", "")
    try:
        return int(shortened_fraction)
    except:
        return np.inf


def get_protein2num_peptides_proteingroups_df(protein_groups_file, evidence_file):
    protein_groups_df = pd.read_csv(protein_groups_file, sep = "\t", usecols=["Protein IDs", "Evidence IDs"])
    print(evidence_file)
    set_of_valid_evidence_ids = get_set_of_valid_evidence_ids(evidence_file)
    protein2num_peptides = {}
    for _, row in protein_groups_df.iterrows():
        evidence_ids = row["Evidence IDs"].split(";")
        evidence_ids = [x for x in evidence_ids if x in set_of_valid_evidence_ids]
        protid = row["Protein IDs"]
        protein2num_peptides[protid] = len(evidence_ids)
    return protein2num_peptides

def get_set_of_valid_evidence_ids(evidence_file):
    evidence_df = pd.read_csv(evidence_file, usecols=["id", "Intensity", "Sequence", "Modified sequence","Charge", "Raw file"], sep="\t")
    evidence_df = evidence_df[~evidence_df["Intensity"].isna()]
    return set(evidence_df["id"].astype('str'))


def get_protein2numpeptides_ion_df(ion_df):
    ion_df = ion_df.reset_index()
    numeric_columns = ion_df.select_dtypes(include=np.number).columns
    protein2numpeptides = {}
    for protid, df in ion_df.groupby("protein"):
        df_numeric = df[numeric_columns]
        #get number of non nan values in dataframe
        num_non_nan = df_numeric.count().sum()
        protein2numpeptides[protid] = num_non_nan
    return protein2numpeptides


list_of_protids_to_check = ['P35268', 'P33993;P33993-3;P33993-2', "Q9NQS1", "O95714;Q9BVR0", "Q9GZU8", "Q13510-2;Q13510;Q13510-3", "Q9NQS1", "P54886;P54886-2", "Q9GZU8", "Q9UP83-2;Q9UP83;Q9UP83-3"]

compare_number_of_datapoints_in_traces_w_protgroup_id_mapping(aq_reformatted_file, protein_groups_file_double, annotated_file)

../../test_data/system_tests/quicktests/mq_evidence/shortened_input.tsv.protgroup_annotated.tsv
protid: A1X283;G3V144;Q5TCZ1;H0Y507;Q5TCZ1-3, num_ions_iondf: 60, num_ions_proteingroupsdf: 60
protid: A6NDG6;H3BV17, num_ions_iondf: 169, num_ions_proteingroupsdf: 169
protid: A6NHR9;A6NHR9-2;J3KTL8;A6NHR9-3, num_ions_iondf: 140, num_ions_proteingroupsdf: 140
protid: A6NIH7, num_ions_iondf: 18, num_ions_proteingroupsdf: 18
protid: G3V150;O94766-2;O94766;E9PNA1;REV__A0A0C4DGF1;REV__Q9Y2Y4;REV__Q9BRK4, num_ions_iondf: 24, num_ions_proteingroupsdf: 24
protid: O00231;O00231-2;J3QRY4;J3QS13;J3KSW3, num_ions_iondf: 450, num_ions_proteingroupsdf: 450
protid: O15357;A0A0A0MTP6;O15357-2;F5GY16;H7BXR2;Q92835;H0YFZ4;Q92835-2;Q7Z7G1;F5H588;D6RJB9;A0A0A0MSJ4;H0Y5Q9;Q7Z7G1-3;Q92835-3, num_ions_iondf: 268, num_ions_proteingroupsdf: 268
protid: O43324;D6RBD7;C9J1V9;O43324-2;H0YAL7;D6RCQ0;H0YAH9, num_ions_iondf: 107, num_ions_proteingroupsdf: 107
protid: O75489;E9PS48;O75489-2;E9PJE7;E9PKL8;G3V194, num_ions

# Learning Tests

In [5]:
import seaborn as sns
petal_df = sns.load_dataset("iris")
petal_df = petal_df.drop_duplicates(subset=["sepal_length"]).iloc[:10]
display(petal_df)
surjective_mapping_df = pd.DataFrame({"species" : petal_df["species"], "sepal_length" : petal_df["sepal_length"]})
#add one more row to the surjective mapping
surjective_mapping_df = surjective_mapping_df.append({"species" : "newspecies", "sepal_length" : 5.1}, ignore_index=True)
display(surjective_mapping_df)

merged_df = petal_df.merge(surjective_mapping_df, left_on="sepal_length", right_on = "sepal_length", how="right", suffixes=("", "_right"))
display(merged_df)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
8,4.4,2.9,1.4,0.2,setosa
11,4.8,3.4,1.6,0.2,setosa
13,4.3,3.0,1.1,0.1,setosa
14,5.8,4.0,1.2,0.2,setosa


/var/folders/wb/j08kzzbd2hd8838tn3ll0jfm0000gn/T/ipykernel_9893/984504079.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  surjective_mapping_df = surjective_mapping_df.append({"species" : "newspecies", "sepal_length" : 5.1}, ignore_index=True)


,species,sepal_length
0,setosa,5.1
1,setosa,4.9
2,setosa,4.7
3,setosa,4.6
4,setosa,5.0
5,setosa,5.4
6,setosa,4.4
7,setosa,4.8
8,setosa,4.3
9,setosa,5.8


,sepal_length,sepal_width,petal_length,petal_width,species,species_right
0,5.1,3.5,1.4,0.2,setosa,setosa
1,4.9,3.0,1.4,0.2,setosa,setosa
2,4.7,3.2,1.3,0.2,setosa,setosa
3,4.6,3.1,1.5,0.2,setosa,setosa
4,5.0,3.6,1.4,0.2,setosa,setosa
5,5.4,3.9,1.7,0.4,setosa,setosa
6,4.4,2.9,1.4,0.2,setosa,setosa
7,4.8,3.4,1.6,0.2,setosa,setosa
8,4.3,3.0,1.1,0.1,setosa,setosa
9,5.8,4.0,1.2,0.2,setosa,setosa
